<font color=red>注：此处是文档第62页</font>

## PyTorch之小试牛刀

### 1 PyTorch的核心是两个主要特征：
- 一个n维张量，类似于numpy，但可以在GPU上运行
- 搭建和训练神经网络时的自动微分/求导机制

本章节我们将使用全连接的ReLU网络作为运行示例。该网络将有一个单一的隐藏层，并将使用梯度下降训练，通过最小化网络输出和真正结果的欧几里得距离，来拟合随机生成的数据。

### 2.张量
#### 2.1 热身: Numpy
在介绍PyTorch之前，本章节将首先使用numpy实现网络。 Numpy提供了一个n维数组对象，以及许多用于操作这些数组的 函数。Numpy是用于科学计算的通用框架;它对计算图、深度学习和梯度一无所知。然而，我们可以很容易地使用NumPy，手动实现网络的前向和反向传播，来拟合随机数据：

In [1]:
import numpy as np

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 100维的特征64条
x = np.random.randn(N, D_in)
# 10维的目标64条
y = np.random.randn(N, D_out)

# 权重简化了，没有加b
# 隐含层的权重1000条，100维用于
w1 = np.random.randn(D_in, H)
# 输出层权重100条，10维，代表将100维的数据降到10维上
w2 = np.random.randn(H, D_out)

# 学习速率
learning_rate = 1e-6
# 模型迭代500次
for t in range(500):
    # 前向传递：计算预测值y
    # 隐含层
    h = x.dot(w1)
    # relu激活函数
    h_relu = np.maximum(h, 0)
    # 输出层
    y_pred = h_relu.dot(w2)
    
    # 损失（方差）
    loss = np.square(y_pred - y).sum()
    print(t, loss)
    
    # 反向传播，计算w1和w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 21625017.786978107
1 15492735.284744546
2 11820344.697453279
3 9203524.791954953
4 7156994.467656947
5 5527328.389814605
6 4232306.456358977
7 3229062.39203347
8 2464440.966111377
9 1892549.3348155867
10 1466281.20584617
11 1149248.8735810579
12 912335.0979639785
13 734031.3241710487
14 598287.492205212
15 493698.90405736025
16 412080.24064294074
17 347612.60714092234
18 296044.6806843659
19 254293.8580420378
20 220104.56688590907
21 191841.28064131117
22 168272.5336866175
23 148416.94612215576
24 131561.43416460362
25 117152.93887829751
26 104746.46838411085
27 93994.84167170573
28 84627.48832406363
29 76422.78582652539
30 69201.77777383415
31 62818.63988303395
32 57154.754977534074
33 52113.9958648186
34 47616.958475896325
35 43584.90259236867
36 39962.341721096556
37 36698.85766671812
38 33751.86165953751
39 31084.92576331173
40 28666.132806948895
41 26467.42885109051
42 24466.718795116365
43 22644.6694015229
44 20980.505367577138
45 19458.63539391223
46 18065.137065113384
47 1678

367 0.01522019506786878
368 0.01466226321044569
369 0.014124858299828583
370 0.01360717167571987
371 0.013108518410098446
372 0.012628215722594517
373 0.012165613195948843
374 0.011719929604542514
375 0.011290619311719003
376 0.01087708463699441
377 0.010478779094817599
378 0.010095058839296261
379 0.009725462002696616
380 0.009369397196362058
381 0.009026444216411269
382 0.008696039934984258
383 0.008377770208883039
384 0.008071186894019752
385 0.007775887910009523
386 0.0074913774946672364
387 0.0072172960000918595
388 0.00695326707459808
389 0.006698940002512133
390 0.006453935121356747
391 0.006217937516714352
392 0.0059905531133868616
393 0.005771513808963263
394 0.0055605041007215405
395 0.005357213526497442
396 0.0051613771745955385
397 0.0049727298521956235
398 0.004790995584869493
399 0.004615897785941404
400 0.004447213533029442
401 0.0042847104537631665
402 0.004128188686709642
403 0.00397737347383186
404 0.003832076850285429
405 0.0036920942130694353
406 0.00355725023498119

#### 2.2 PyTorch：张量
Numpy是一个很棒的框架，但它不能利用GPU来加速其数值计算。 对于现代深度神经网络，GPU通常提供50倍或更高的加速，所以，numpy不能满足当代深度学习的需求。

在这里，先介绍最基本的PyTorch概念：

**张量（Tensor）**：PyTorch的tensor在概念上与numpy的array相同： tensor是一个n维数组，PyTorch提供了许多函数用于操作这些张量。任何希望使用NumPy执行的计算也可以使用PyTorch的tensor来完成，可以认为它们是科学计算的通用工具。

与Numpy不同，PyTorch可以利用GPU加速其数值计算。要在GPU上运行Tensor,在构造张量使用device参数把tensor建立在GPU上。

在这里，本章使用tensors将随机数据上训练一个两层的网络。和前面NumPy的例子类似，我们使用PyTorch的tensor，手动在网络中实现前向传播和反向传播：

In [2]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda: 0")  # 在GPU上运行

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)
    
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 29860810.0
1 25149770.0
2 23565784.0
3 21608952.0
4 18082518.0
5 13376106.0
6 8945960.0
7 5591560.0
8 3467792.25
9 2216302.0
10 1503185.375
11 1086516.0
12 831673.1875
13 664995.6875
14 548560.375
15 462262.375
16 395220.25
17 341353.0
18 297054.96875
19 259997.328125
20 228582.46875
21 201790.984375
22 178811.71875
23 158976.71875
24 141756.46875
25 126737.2109375
26 113601.5
27 102093.328125
28 91954.4765625
29 83004.9921875
30 75077.421875
31 68041.9609375
32 61780.76953125
33 56198.1796875
34 51204.69140625
35 46729.01171875
36 42708.4453125
37 39088.34375
38 35824.3203125
39 32876.11328125
40 30208.10546875
41 27790.708984375
42 25597.634765625
43 23602.150390625
44 21785.6796875
45 20128.712890625
46 18616.212890625
47 17233.470703125
48 15968.017578125
49 14807.529296875
50 13743.5576171875
51 12766.25390625
52 11867.884765625
53 11040.666015625
54 10278.67578125
55 9576.53515625
56 8928.5380859375
57 8329.9462890625
58 7776.5126953125
59 7263.921875
60 6788.861328125
61 6348.

### 3.自动求导
#### 3.1 PyTorch：张量和自动求导
在上面的例子中，需要手动实现神经网络的前向和后向传递。手动实现反向传递对于小型双层网络来说并不是什么大问题，但对于大型复杂网络来说很快就会变得非常繁琐。

但是可以使用自动微分来自动计算神经网络中的后向传递。 PyTorch中的 `autograd` 包提供了这个功能。当使用`autograd`时，网络前向传播将定义一个计算图；图中的节点是`tensor`，边是函数，这些函数是输出`tensor`到输入`tensor`的映射。这张计算图使得在网络中反向传播时梯度的计算十分简单。

这听起来很复杂，在实践中使用起来非常简单。 如果我们想计算某些的`tensor`的梯度，我们只需要在建立这个`tensor`时加入这么一句：`requires_grad=True`。这个`tensor`上的任何PyTorch的操作都将构造一个计算图，从而允许我们稍后在图中执行反向传播。如果这个`tensor x `的`requires_grad=True`，那么反向传播之后`x.grad`将会是另一个张量，其为x关于某个标量值的梯度。

有时可能希望防止PyTorch在`requires_grad=True`的张量执行某些操作时构建计算图；例如，在训练神经网络时，我们通常不希望通过权重更新步骤进行反向传播。在这种情况下，我们可以使用`torch.no_grad()`上下文管理器来防止构造计算图。

下面我们使用PyTorch的`Tensors`和`autograd`来实现我们的两层的神经网络；我们不再需要手动执行网络的反向传播：

In [3]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda: 0")  # 在GPU上运行

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 设置requires_grad = False表示我们不需要计算梯度
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 设置requires_grad = True表示我们想要计算梯度
# 在向后传球期间计算这些张量
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：使用tensors上的操作计算预测值y;
    # 由于w1和w2有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # 使用Tensors上的操作计算和打印丢失。
    # loss是一个形状为()的张量
    # loss.item() 得到这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # 使用autograd计算反向传播。这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()
    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 35478504.0
1 35700004.0
2 37935332.0
3 35161188.0
4 25915434.0
5 14833444.0
6 7291321.0
7 3576056.5
8 1998398.25
9 1310648.25
10 970313.25
11 770070.875
12 633887.0
13 532097.5
14 451920.0625
15 386881.28125
16 333347.59375
17 288738.375
18 251246.609375
19 219541.78125
20 192572.09375
21 169528.78125
22 149763.203125
23 132716.75
24 117956.9765625
25 105125.59375
26 93940.25
27 84149.328125
28 75550.7890625
29 67984.09375
30 61302.7890625
31 55385.4921875
32 50129.4765625
33 45452.3984375
34 41282.93359375
35 37556.890625
36 34218.98046875
37 31222.533203125
38 28526.71875
39 26103.138671875
40 23916.7578125
41 21939.72265625
42 20151.025390625
43 18527.640625
44 17052.28125
45 15709.9609375
46 14484.35546875
47 13366.048828125
48 12344.3984375
49 11410.5
50 10555.55078125
51 9771.9326171875
52 9053.208984375
53 8393.5390625
54 7787.06591796875
55 7229.15673828125
56 6715.63232421875
57 6242.73681640625
58 5806.59765625
59 5403.82275390625
60 5031.73291015625
61 4687.5751953125
62 4

380 0.0004686604079324752
381 0.000453094020485878
382 0.00043982092756778
383 0.00042663380736485124
384 0.0004137512587476522
385 0.000401211524149403
386 0.00038973416667431593
387 0.0003782229032367468
388 0.00036827076110057533
389 0.0003572218702174723
390 0.0003467483911663294
391 0.0003377805114723742
392 0.0003284508129581809
393 0.000318407837767154
394 0.0003092239494435489
395 0.0003010405052918941
396 0.00029296957654878497
397 0.00028546820976771414
398 0.00027781783137470484
399 0.00027022886206395924
400 0.0002639323356561363
401 0.00025677774101495743
402 0.0002505457669030875
403 0.00024387570738326758
404 0.0002379792567808181
405 0.00023213523672893643
406 0.00022602843819186091
407 0.00022096723841968924
408 0.0002152019733330235
409 0.00020982595742680132
410 0.00020491024770308286
411 0.00019981116929557174
412 0.00019452458946034312
413 0.0001897248876048252
414 0.00018602798809297383
415 0.0001814886782085523
416 0.00017706240760162473
417 0.0001731318770907819

#### 3.2 PyTorch：定义新的自动求导函数
在底层，每一个原始的自动求导运算实际上是两个在Tensor上运行的函数。其中，**`forward`** 函数计算从输入Tensors获得的输出Tensors。而**`backward`** 函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。

在PyTorch中，我们可以很容易地通过定义 **`torch.autograd.Function`** 的子类并实现forward和backward函数，来定义自己的自动求导运算。之后我们就可以使用这个新的自动梯度运算符了。然后，我们可以通过构造一个实例并像调用函数一样，传入包含输入数据的tensor调用它，这样来使用新的自动求导运算。

这个例子中，我们自定义一个自动求导函数来展示`ReLU`的非线性。并用它实现我们的两层网络：

In [4]:
import torch

class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，
    并完成张量的正向和反向传播。
    """
    
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，
        并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(x)
        return x.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone()
        grad_x[x<0] = 0
        return grad_x
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 正向传播：使用张量上的操作来计算输出值y；
    # 我们通过调用 MyReLU.apply 函数来使用自定义的ReLU
    y_pred = MyReLU.apply(x.mm(w1)).mm(w2)
    
    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 32327278.0
1 29030792.0
2 31113542.0
3 33050398.0
4 30759384.0
5 22991768.0
6 13896627.0
7 7216564.5
8 3702307.75
9 2070463.625
10 1330166.0
11 963203.9375
12 755259.8125
13 619546.0
14 520902.90625
15 444227.71875
16 382296.15625
17 331174.6875
18 288392.15625
19 252232.984375
20 221480.859375
21 195184.46875
22 172570.296875
23 153028.875
24 136084.171875
25 121349.96875
26 108480.90625
27 97204.609375
28 87298.984375
29 78561.7265625
30 70825.15625
31 63962.4921875
32 57857.1328125
33 52420.671875
34 47573.7421875
35 43235.74609375
36 39349.66796875
37 35862.58203125
38 32725.08984375
39 29897.1484375
40 27342.373046875
41 25035.4921875
42 22944.89453125
43 21049.98828125
44 19330.6953125
45 17767.306640625
46 16343.8173828125
47 15046.0361328125
48 13862.451171875
49 12780.6328125
50 11790.9580078125
51 10885.18359375
52 10055.384765625
53 9294.466796875
54 8596.158203125
55 7955.1669921875
56 7365.7587890625
57 6823.8486328125
58 6324.91064453125
59 5865.21533203125
60 5441.4433

368 0.0002771244617179036
369 0.000269462529104203
370 0.0002614120894577354
371 0.000254364509601146
372 0.00024772531469352543
373 0.00024031831708271056
374 0.00023434922331944108
375 0.00022821445600129664
376 0.00022229092428460717
377 0.00021631279378198087
378 0.00020970650075469166
379 0.00020353318541310728
380 0.00019910623086616397
381 0.00019400729797780514
382 0.00018855312373489141
383 0.00018374909996055067
384 0.00017881429812405258
385 0.00017473984917160124
386 0.00017016383935697377
387 0.0001663657312747091
388 0.00016207955195568502
389 0.00015811380580998957
390 0.00015395949594676495
391 0.00015041857841424644
392 0.000146915961522609
393 0.00014355697203427553
394 0.00013993983156979084
395 0.00013643749116454273
396 0.00013374918489716947
397 0.00013059237971901894
398 0.00012749772577080876
399 0.00012448968482203782
400 0.00012165460066171363
401 0.00011904437269549817
402 0.00011626501509454101
403 0.00011342577636241913
404 0.00011101121344836429
405 0.0001

#### 3.3 TensorFlow：静态图
PyTorch自动求导看起来非常像TensorFlow：这两个框架中，我们都定义计算图，使用自动微分来计算梯度。两者最大的不同就是TensorFlow的计算图是静态的，而PyTorch使用动态的计算图。

在TensorFlow中，我们定义计算图一次，然后重复执行这个相同的图，可能会提供不同的输入数据。而在PyTorch中，每一个前向通道定义一个新的计算图。

静态图的好处在于你可以预先对图进行优化。例如，一个框架可能要融合一些图的运算来提升效率，或者产生一个策略来将图分布到多个GPU或机器上。如果重复使用相同的图，那么在重复运行同一个图时，前期潜在的代价高昂的预先优化的消耗就会被分摊开。

静态图和动态图的一个区别是控制流。对于一些模型，我们希望对每个数据点执行不同的计算。例如，一个递归神经网络可能对于每个数据点执行不同的时间步数，这个展开（unrolling）可以作为一个循环来实现。对于一个静态图，循环结构要作为图的一部分。因此，TensorFlow提供了运算符（例如 **`tf.scan`** ）来把循环嵌入到图当中。对于动态图来说，情况更加简单：既然我们为每个例子即时创建图，我们可以使用普通的命令式控制流来为每个输入执行不同的计算。

为了与上面的PyTorch自动梯度实例做对比，我们使用TensorFlow来拟合一个简单的2层网络：

In [10]:
import tensorflow.compat.v1 as tf  # v2调用v1
import numpy as np

# 首先我们建立计算图（computational graph）
# N是批大小；D是输入维度；
# H是隐藏层维度；D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10
# 为输入和目标数据创建placeholder；
# 当执行计算图时，他们将会被真实的数据填充
tf.disable_eager_execution()  # v2调用v1
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

# 为权重创建Variable并用随机数据初始化
# TensorFlow的Variable在执行计算图时不会改变
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

# 前向传播：使用TensorFlow的张量运算计算预测值y。
# 注意这段代码实际上不执行任何数值运算；
# 它只是建立了我们稍后将执行的计算图。
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

loss = tf.reduce_sum((y - y_pred) ** 2.0)

gred_w1, gred_w2 = tf.gradients(loss, [w1, w2])

# 使用梯度下降更新权重。为了实际更新权重，我们需要在执行计算图时计算new_w1和new_w2。
# 注意，在TensorFlow中，更新权重值的行为是计算图的一部分;
# 但在PyTorch中，这发生在计算图形之外。
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * gred_w1)
new_w2 = w2.assign(w2 - learning_rate * gred_w2)

# 现在我们搭建好了计算图，所以我们开始一个TensorFlow的会话（session）来实际执行计算图。
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    
    for _ in range(500):
        # 多次运行计算图。每次执行时，我们都用feed_dict参数，
        # 将x_value绑定到x，将y_value绑定到y，
        # 每次执行图形时我们都要计算损失、new_w1和new_w2；
        # 这些张量的值以numpy数组的形式返回。
        loss_value, _, _ = sess.run([loss, new_w1, new_w2], feed_dict={x: x_value, y: y_value})
        print(loss_value)

27156216.0
21372004.0
19332348.0
18163022.0
16454443.0
13806622.0
10585867.0
7507474.0
5063453.5
3355118.5
2249210.0
1557442.5
1125925.2
851215.2
670157.75
545094.56
454846.12
386648.56
333053.5
289689.8
253849.81
223769.38
198123.03
176098.6
157072.97
140498.81
125975.21
113218.945
101952.555
91971.44
83106.0
75211.945
68168.24
61873.766
56235.594
51175.1
46626.562
42529.645
38833.2
35494.387
32474.05
29737.893
27256.273
25005.285
22959.838
21099.098
19404.453
17859.324
16448.838
15160.047
13981.578
12902.82
11914.732
11008.682
10177.463
9414.705
8713.861
8070.02
7478.565
6934.037
6432.6357
5970.461
5543.931
5150.191
4786.5063
4450.4023
4139.6074
3852.0166
3585.7832
3339.1902
3110.7825
2899.0444
2702.7004
2520.484
2351.3901
2194.4011
2048.5088
1912.9508
1786.896
1669.6343
1560.5437
1458.9979
1364.4568
1276.3977
1194.3447
1117.8553
1046.5322
980.0181
917.9496
860.039
805.9581
755.45435
708.27686
664.19763
622.998
584.4893
548.4656
514.7687
483.23407
453.72595
426.10883
400.24832
376.02

### 4.nn模块
#### 4.1 PyTorch： nn
计算图和autograd是十分强大的工具，可以定义复杂的操作并自动求导；然而对于大规模的网络，autograd太过于底层。 在构建神经网络时，我们经常考虑将计算安排成层，其中一些具有可学习的参数，它们将在学习过程中进行优化。

TensorFlow里，有类似Keras，TensorFlow-Slim和TFLearn这种封装了底层计算图的高度抽象的接口，这使得构建网络十分方便。

在PyTorch中，包 nn 完成了同样的功能。nn包中定义一组大致等价于层的模块。一个模块接受输入的tesnor，计算输出的tensor，而且还保存了一些内部状态比如需要学习的tensor的参数等。nn包中也定义了一组损失函数（loss functions），用来训练神经网络。

这个例子中，我们用nn包实现两层的网络：

In [13]:
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)
    
    # 计算并打印损失。
    # 传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    # 反向传播之前清零梯度
    model.zero_grad()
    
    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()
    
    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 682.0833129882812
1 629.7072143554688
2 584.7716064453125
3 545.7764282226562
4 511.34967041015625
5 480.9350891113281
6 453.54339599609375
7 428.3785095214844
8 405.0462951660156
9 383.2603759765625
10 362.87591552734375
11 343.6944885253906
12 325.53564453125
13 308.3944091796875
14 292.17645263671875
15 276.74456787109375
16 262.052001953125
17 248.0528106689453
18 234.63259887695312
19 221.86102294921875
20 209.660888671875
21 198.005615234375
22 186.88026428222656
23 176.27105712890625
24 166.1709442138672
25 156.55960083007812
26 147.43885803222656
27 138.78321838378906
28 130.5286102294922
29 122.70169067382812
30 115.31281280517578
31 108.31902313232422
32 101.71699523925781
33 95.48482513427734
34 89.62298583984375
35 84.09318542480469
36 78.89803314208984
37 74.00912475585938
38 69.42298126220703
39 65.12841796875
40 61.10314178466797
41 57.3228759765625
42 53.788116455078125
43 50.47406768798828
44 47.36839294433594
45 44.471927642822266
46 41.76225280761719
47 39.22714996

381 5.529787813429721e-05
382 5.3483636293094605e-05
383 5.17322841915302e-05
384 5.004406193620525e-05
385 4.8410569434054196e-05
386 4.682647704612464e-05
387 4.52976455562748e-05
388 4.38178212789353e-05
389 4.2388419387862086e-05
390 4.1005801904248074e-05
391 3.966478107031435e-05
392 3.837557596853003e-05
393 3.712387115228921e-05
394 3.5913646570406854e-05
395 3.4743363357847556e-05
396 3.361469498486258e-05
397 3.251868474762887e-05
398 3.146374365314841e-05
399 3.044107143068686e-05
400 2.9452978196786717e-05
401 2.8496779123088345e-05
402 2.7570607926463708e-05
403 2.6678097128751688e-05
404 2.5809327780734748e-05
405 2.4974788175313734e-05
406 2.4163768102880567e-05
407 2.338152080483269e-05
408 2.2622481992584653e-05
409 2.1891595679335296e-05
410 2.1181831471039914e-05
411 2.0498646335909143e-05
412 1.9834349586744793e-05
413 1.9192739273421466e-05
414 1.8572542103356682e-05
415 1.797209188225679e-05
416 1.7391568690072745e-05
417 1.6831583707244135e-05
418 1.6288840924971